# 使用FP-growth算法来发现频繁项集
---
使用 FP-growth算法更有效的挖掘`频繁项集`, 它只需要对数据库进行两次扫描, 而Apriori算法对于每个潜在的频繁项集都会扫描数据集判定给定模式是否频繁.
FP-growth算法将数据存储在一种称为FP树的紧凑数据结构中.FP代表频繁模式(Frequent Pattern)

使用实例: 搜索引擎自动补全查询词项.

基本过程:
- 构建FP树
- 从FP树中挖掘频繁项集

In [56]:
class TreeNode:
    def __init__(self, name, counts, parent):
        self.name = name  # 节点名称
        self.counts = counts  # 节点出现次数
        self.parent = parent  # 父节点
        self.children = {}  # 子节点
        # 节点链接
        self.node_link = None  # 用于连接 不同项集 的 相同的项
    
    def inc(self, counts):
        """
        增加项出现的次数
        """
        self.counts += counts
    
    def __repr__(self):
        return f"Node({self.name}: {self.counts})"
    
    def disp(self, index=1):
        """
        文本方式显示树
        """
        print(' ' * index, self.name, ' ', self.counts)
        for child in self.children.values():
            child.disp(index+1)

## FP-growth 原理
[构建FP树](https://github.com/apachecn/AiLearning/blob/master/docs/ml/12.%E4%BD%BF%E7%94%A8FP-growth%E7%AE%97%E6%B3%95%E6%9D%A5%E9%AB%98%E6%95%88%E5%8F%91%E7%8E%B0%E9%A2%91%E7%B9%81%E9%A1%B9%E9%9B%86.md#fp-growth-%E5%8E%9F%E7%90%86)

In [40]:
def load_data():
    sample = [
        ['r', 'z', 'h', 'j', 'p'],
       ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
       ['z'],
       ['r', 'x', 'n', 'o', 's'],
    #    ['r', 'x', 'n', 'o', 's'],
       ['y', 'r', 'x', 'z', 'q', 't', 'p'],
       ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']
    ]
    return sample

In [41]:
data = load_data()

In [58]:
class FPTree:
    def __init__(self, min_support = 1):
        # 出现的最少次数 低于此值的项会被丢弃
        self.min_support = min_support
        self.root = None
    
    def fit(self, data):
        data = self.init_data(data)
        self.header_table = {}
        # 1. 遍历所有的数据集合，计算所有项的支持度。
        for set_ in data:
            for item in set_:
                # set_ : frozenset(['z', 'y', 'x', 'w', 'v', 'u', 't', 's'])
                # 'z': 1(z累计次数) + 1(当前set出现次数) 
                self.header_table[item] = self.header_table.get(item, 0) + data[set_]
       
        # 2. 丢弃非频繁的项
        for key in list(self.header_table.keys()):  # 需要创建一个新的list
            if self.header_table[key] < self.min_support:
                del self.header_table[key]
#         print(head_table)
        if not self.header_table:
            return None, None
        freq_sets = set(self.header_table.keys())
        
        # 3. 基于 支持度 降序排序所有的项。
        sorted_list = sorted(self.header_table.items(), key=lambda item: item[1], reverse=True)
        
        # 4. 所有数据集合按照得到的顺序重新整理
        # 5. 重新整理完成后，丢弃每个集合末尾非频繁的项。 
        print(sorted_list)
        new_keys = []
        for set_, freqs in data.items():
            new_keys.append([item[0] for item in sorted_list if item[0] in set_])
        print(new_keys)  # ['z', 'r'], ['z', 'x', 'y', 's', 't'], ['z']
        
        for k in self.header_table:
            # 格式化： dict{元素key: [元素次数, None]}
            self.header_table[k] = [self.header_table[k], None]
        
        # 6. 读取每个集合插入FP树中，同时用一个头部链表数据结构维护不同集合的相同项
        self.root = TreeNode('Null', 1, None)
        for key, count, new_key in zip(data.keys(), data.values(), new_keys):
            # 原始的frozenset, 项集合出现次数, 整理并舍弃末尾的集合
            self.update_tree(self.root, new_key, self.header_table, count)
    
    def update_header(self, node_to_test, target_node):
        """updateHeader(更新头指针，建立相同元素之间的关系，例如： 左边的r指向右边的r值，就是后出现的相同元素 指向 已经出现的元素)
        从头指针的nodeLink开始，一直沿着nodeLink直到到达链表末尾。这就是链表。
        性能：如果链表很长可能会遇到迭代调用的次数限制。
        Args:
            nodeToTest  满足minSup {所有的元素+(value, treeNode)}
            targetNode  Tree对象的子节点
        """
        # 建立相同元素之间的关系，例如： 左边的r指向右边的r值
        while (node_to_test.node_link is not None):
            node_to_test = node_to_test.node_link
        node_to_test.node_link = target_node
    
    def update_tree(self, root, new_keys, header_table, count):
        # 递归形式 完成FP树
        # print(root.children)
        if new_keys[0] in root.children:  # 取最前面一个(出现次数最多的)
            # 如果该元素在 inTree.children 这个字典中，就进行累加
            root.children[new_keys[0]].inc(count)
        else:
            # 如果不存在子节点，我们为该inTree添加子节点
            root.children[new_keys[0]] = TreeNode(new_keys[0], count, root)
            # 如果满足minSup的dict字典的value值第二位为null， 我们就设置该元素为 本节点对应的tree节点
            # 如果元素第二位不为null，我们就更新header节点, header_table表一项只对应一个节点
            if header_table[new_keys[0]][1] is None:
                header_table[new_keys[0]][1] = root.children[new_keys[0]]
            else:
                self.update_header(header_table[new_keys[0]][1], root.children[new_keys[0]])
        if len(new_keys)>1:
            self.update_tree(root.children[new_keys[0]], new_keys[1:], header_table, count)
        
    
    def init_data(self, data):
        ret_dict = {}  # {frozenset() : 次数}
        for item in data:
            ret_dict[frozenset(item)] = ret_dict.get(frozenset(item), 0) + 1
        return ret_dict

In [59]:
fp = FPTree(min_support=3)
fp.fit(data)

[('z', 5), ('x', 4), ('r', 3), ('t', 3), ('s', 3), ('y', 3)]
[['z', 'r'], ['z', 'x', 't', 's', 'y'], ['z'], ['x', 'r', 's'], ['z', 'x', 'r', 't', 'y'], ['z', 'x', 't', 's', 'y']]


In [60]:
fp.header_table

{'z': [5, Node(z: 5)],
 'r': [3, Node(r: 1)],
 'x': [4, Node(x: 3)],
 't': [3, Node(t: 2)],
 's': [3, Node(s: 2)],
 'y': [3, Node(y: 2)]}

In [61]:
fp.root.disp()

  Null   1
   z   5
    r   1
    x   3
     t   2
      s   2
       y   2
     r   1
      t   1
       y   1
   x   1
    r   1
     s   1


In [14]:
d1 = {'a':2, 'b':2}
l1 = [[2], []]
for a, b ,c in zip(d1.keys(), d1.values(), l1):
    print(a, b, c)

a 2 [2]
b 2 []
